In [11]:
from utils import load_and_preprocess_data
from model import MiniGPT
import tiktoken
import torch
import numpy as np

tokenizer = tiktoken.get_encoding("gpt2")

vocab_size = tokenizer.n_vocab
num_transformer_blocks = 4
maxlen = 256
embed_dim = 256
num_heads = 8
feed_forward_dim = 256
batch_size = 144 
num_epochs = 1
top_k = 10

text_dl = load_and_preprocess_data('TinyStories-train.txt', batch_size, maxlen, num_epochs)


In [12]:
def train_step(model,optimizer,inputs,targets):
    logits = model(inputs)

    #Torch black magic
    B, L, V = logits.shape
    logits = logits.view(B * L, V)
    targets = targets.view(B * L)

    optimizer.zero_grad()
    loss = torch.nn.functional.cross_entropy(input=logits,target=targets)
    loss.backward()
    optimizer.step()
    return loss


In [ ]:
model = MiniGPT(
    maxlen=maxlen,
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    num_heads=num_heads,
    feed_forward_dim=feed_forward_dim,
    num_transformer_blocks=num_transformer_blocks,
    tokenizer=tokenizer,
    top_k=top_k
)

optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

prep_target_batch = torch.vmap(
    lambda tokens: torch.cat((tokens[1:], tokens.new_zeros(1)), dim=0)
)

: 

In [ ]:
step = 0
for batch in text_dl:
    input_batch = torch.tensor(np.array(batch).T)
    target_batch = prep_target_batch(input_batch)
    loss = train_step(model,optimizer,input_batch,target_batch)

    if (step+1) % 20 == 0:
        print(loss)
    step+=1
    